In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from tobit_model import TobitRegression

In [2]:
dat = pd.read_csv("https://stats.idre.ucla.edu/stat/data/tobit.csv")

In [3]:
dat['proggeneral'] = np.where(dat['prog'] == 'general', 1, 0)
dat['progvocational'] = np.where(dat['prog'] == 'vocational', 1, 0)

In [4]:
dat.describe()

,id,read,math,apt,proggeneral,progvocational
count,200.000000,200.000000,200.000000,200.00000,200.000000,200.000000
mean,100.500000,52.230000,52.645000,640.03500,0.525000,0.250000
std,57.879185,10.252937,9.368448,99.21903,0.500628,0.434099
min,1.000000,28.000000,33.000000,352.00000,0.000000,0.000000
25%,50.750000,44.000000,45.000000,575.50000,0.000000,0.000000
50%,100.500000,50.000000,52.000000,633.00000,1.000000,0.000000
75%,150.250000,60.000000,59.000000,705.25000,1.000000,0.250000
max,200.000000,76.000000,75.000000,800.00000,1.000000,1.000000


In [5]:
tr = TobitRegression(dat['apt'], sm.add_constant(dat[['read', 'math', 'proggeneral', 'progvocational']]),(-np.inf, 800)).fit()
tr.summary()
# https://m-clark.github.io/models-by-example/tobit.html

<class 'statsmodels.iolib.summary.Summary'>
"""
                           TobitRegression Results                            
==============================================================================
Dep. Variable:                    apt   Log-Likelihood:                -1041.1
Model:                TobitRegression   AIC:                             2092.
Method:            Maximum Likelihood   BIC:                             2109.
Date:                Thu, 28 Dec 2023                                         
Time:                        01:08:14                                         
No. Observations:                 200                                         
Df Residuals:                     195                                         
Df Model:                           4                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            209.5970     32.698      6.410      0.000     145.109     274.085
read               2.6977      0.619      4.361      0.000       1.478       3.918
math               5.9140      0.709      8.343      0.000       4.516       7.312
proggeneral      -12.7083     12.404     -1.025      0.307     -37.172      11.755
progvocational   -46.1415     13.721     -3.363      0.001     -73.202     -19.081
==================================================================================
"""

In [6]:
dat['apt2'] = np.where(dat['apt'] < 500, 500, dat['apt'])

In [7]:
tr = TobitRegression(dat['apt2'], sm.add_constant(dat[['read', 'math', 'proggeneral', 'progvocational']]), (400, np.inf)).fit()
tr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           TobitRegression Results                            
==============================================================================
Dep. Variable:                   apt2   Log-Likelihood:                -1092.5
Model:                TobitRegression   AIC:                             2195.
Method:            Maximum Likelihood   BIC:                             2211.
Date:                Thu, 28 Dec 2023                                         
Time:                        01:08:14                                         
No. Observations:                 200                                         
Df Residuals:                     195                                         
Df Model:                           4                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const            270.4124     27.550      9.815      0.000     216.078     324.747
read               2.3278      0.533      4.368      0.000       1.277       3.379
math               5.0855      0.602      8.442      0.000       3.897       6.273
proggeneral      -11.3303     10.735     -1.055      0.293     -32.502       9.841
progvocational   -38.6035     11.866     -3.253      0.001     -62.006     -15.201
==================================================================================
"""

# Tobit Regression
  
## Formation
  
\begin{aligned} Y^{*} = X^{'}\beta+\varepsilon \end{aligned}  
  
\begin{aligned} Y = \begin{cases} a, & \mbox{if } Y^{*} \le a \\ Y^{*}, & \mbox{if } a < Y^{*} < b \\ b, & \mbox{if } Y^{*} \ge b \end{cases} \end{aligned}  
  
where,  
- $X=\begin{pmatrix} x_{1} \\ \vdots \\ x_{p} \end{pmatrix}$
- $\beta=\begin{pmatrix} \beta_{1} \\ \vdots \\ \beta_{p} \end{pmatrix}$
- $\varepsilon \sim N(0, \sigma^2)$
  
## Likelihood Function
  
\begin{aligned} L = \prod_{i=1}^n{[I_{i}^{a}\Phi(\frac{a-X_{i}^{'}\beta}{\sigma})] \times [I_{i}^{b}(1-\Phi(\frac{b-X_{i}^{'}\beta}{\sigma}))] \times [(1-I_{i}^{a}-I_{i}^{b})\frac{1}{\sigma}\phi(\frac{y_{i}-X_{i}^{'}\beta}{\sigma})}] \end{aligned}  
  
where,  
- $I_{i}^{a}=\begin{cases} 1, & \mbox{if } y_{i} = a \\ 0, & \mbox{if } y_{i} > a \end{cases}$
- $I_{i}^{b}=\begin{cases} 1, & \mbox{if } y_{i} = b \\ 0, & \mbox{if } y_{i} < b \end{cases}$
- $X_{i}=\begin{pmatrix} x_{i1} \\ \vdots \\ x_{ip} \end{pmatrix}$
- $\phi(\cdot)$ is standard normal probability density function (PDF)
- $\Phi(\cdot)$ is standard normal cumulative density function (CDF)
  
## Log-likelihood Function
  
\begin{aligned} l = \log{L} = \sum_{i=1}^{n}{I_{i}^{a}\log{\Phi(\frac{a-X_{i}^{'}\beta}{\sigma})+I_{i}^{b}\log{(1-\Phi(\frac{b-X_{i}^{'}\beta}{\sigma}))}+(1-I_{i}^{a}-I_{i}^{b})(\log{\phi(\frac{y_{i}-X_{i}^{'}\beta}{\sigma})}-\frac{1}{2}\log{\sigma^2})}} \end{aligned}  
  
## Score Function (First Derivation)
  
\begin{aligned} \frac{\partial{l}}{\partial{\beta}}=\sum_{i=1}^{n}{(-I_{i}^{a}\frac{\phi(\frac{a-X_{i}^{'}\beta}{\sigma})}{\Phi(\frac{a-X_{i}^{'}\beta}{\sigma})}+I_{i}^{b}\frac{\phi(\frac{b-X_{i}^{'}\beta}{\sigma})}{1-\Phi(\frac{b-X_{i}^{'}\beta}{\sigma})}+(1-I_{i}^{a}-I_{i}^{b})\frac{y_{i}-X_{i}^{'}\beta}{\sigma})\frac{X_{i}^{'}}{\sigma}} \end{aligned} 
\begin{aligned} \frac{\partial{l}}{\partial{\sigma^2}}=\frac{1}{2\sigma}\frac{\partial{l}}{\partial{\sigma}}=\sum_{i=1}^{n}{\frac{1}{2\sigma^2}(-I_{i}^{a}\frac{\phi(\frac{a-X_{i}^{'}\beta}{\sigma})}{\Phi(\frac{a-X_{i}^{'}\beta}{\sigma})}\frac{a-X_{i}^{'}\beta}{\sigma}+I_{i}^{b}\frac{\phi(\frac{b-X_{i}^{'}\beta}{\sigma})}{1-\Phi(\frac{b-X_{i}^{'}\beta}{\sigma})}\frac{b-X_{i}^{'}\beta}{\sigma}+(1-I_{i}^{a}-I_{i}^{b})((\frac{y_{i}-X_{i}^{'}\beta}{\sigma})^2-1))} \end{aligned}  
  
## Hessian Matrix (Second Derivation)
  
\begin{aligned} -\frac{\partial{l}}{\partial{\beta}\partial{\beta^{'}}}=\sum_{i=1}^{n}{\frac{X_{i}}{\sigma}(I_{i}^{a}\frac{\phi(\frac{a-X_{i}^{'}\beta}{\sigma})(\Phi(\frac{a-X_{i}^{'}\beta}{\sigma})\frac{a-X_{i}^{'}\beta}{\sigma}+\phi(\frac{a-X_{i}^{'}\beta}{\sigma}))}{(\Phi(\frac{a-X_{i}^{'}\beta}{\sigma}))^{2}}-I_{i}^{b}\frac{\phi(\frac{b-X_{i}^{'}\beta}{\sigma})((1-\Phi(\frac{b-X_{i}^{'}\beta}{\sigma}))\frac{b-X_{i}^{'}\beta}{\sigma}-\phi(\frac{b-X_{i}^{'}\beta}{\sigma}))}{(1-\Phi(\frac{b-X_{i}^{'}\beta}{\sigma}))^{2}}+(1-I_{i}^{a}-I_{i}^{b}))\frac{X_{i}^{'}}{\sigma}} \end{aligned} 
  
## Prediction
\begin{aligned} E(Y|X)=\Phi(\frac{a-X_{i}^{'}\beta}{\sigma})\times a+(1-\Phi(\frac{b-X_{i}^{'}\beta}{\sigma}))\times b+(\Phi(\frac{b-X_{i}^{'}\beta}{\sigma})-\Phi(\frac{a-X_{i}^{'}\beta}{\sigma}))\times (X_{i}^{'}\beta-\sigma\frac{\phi(\frac{b_{i}-X_{i}^{'}\beta}{\sigma})-\phi(\frac{a_{i}-X_{i}^{'}\beta}{\sigma})}{\Phi(\frac{b-X_{i}^{'}\beta}{\sigma})-\Phi(\frac{a-X_{i}^{'}\beta}{\sigma})})\end{aligned}